# Exploration des Données - MovieLens 20M

**Auteur:** Dady Akrou Cyrille  
**Email:** cyrilledady0501@gmail.com  
**Date:** Décembre 2024

Ce notebook explore le dataset MovieLens 20M pour comprendre la structure des données et identifier les patterns pour notre système de recommandation.

In [ ]:
# Imports nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
import yaml
from datetime import datetime

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Configuration des graphiques
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# Chargement de la configuration
config_path = Path('../config/config.yaml')

with open(config_path, 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

# Chemins des données
data_path = Path(config['data']['base_path'])
dataset_files = config['data']['dataset_files']

print("Configuration chargée avec succès!")
print(f"Chemin des données: {data_path}")

## 1. Chargement des Données

In [ ]:
# Fonction pour charger les données
def load_data():
    """Charge tous les fichiers du dataset MovieLens"""
    data = {}
    
    try:
        # Chargement des ratings
        print("Chargement des ratings...")
        data['ratings'] = pd.read_csv(data_path / dataset_files['ratings'])
        print(f"✓ Ratings: {data['ratings'].shape}")
        
        # Chargement des films
        print("Chargement des films...")
        data['movies'] = pd.read_csv(data_path / dataset_files['movies'])
        print(f"✓ Films: {data['movies'].shape}")
        
        # Chargement des tags
        print("Chargement des tags...")
        data['tags'] = pd.read_csv(data_path / dataset_files['tags'])
        print(f"✓ Tags: {data['tags'].shape}")
        
        # Chargement des liens
        print("Chargement des liens...")
        data['links'] = pd.read_csv(data_path / dataset_files['links'])
        print(f"✓ Liens: {data['links'].shape}")
        
        # Chargement des genome scores (optionnel)
        try:
            print("Chargement des genome scores...")
            data['genome_scores'] = pd.read_csv(data_path / dataset_files['genome_scores'])
            print(f"✓ Genome Scores: {data['genome_scores'].shape}")
        except FileNotFoundError:
            print("⚠️ Genome scores non trouvé")
        
        # Chargement des genome tags (optionnel)
        try:
            print("Chargement des genome tags...")
            data['genome_tags'] = pd.read_csv(data_path / dataset_files['genome_tags'])
            print(f"✓ Genome Tags: {data['genome_tags'].shape}")
        except FileNotFoundError:
            print("⚠️ Genome tags non trouvé")
            
    except Exception as e:
        print(f"❌ Erreur lors du chargement: {e}")
        return None
    
    return data

# Chargement des données
data = load_data()

## 2. Exploration des Ratings

In [ ]:
if data and 'ratings' in data:
    ratings = data['ratings']
    
    print("=== ANALYSE DES RATINGS ===")    print(f"Nombre total de ratings: {len(ratings):,}")    print(f"Nombre d'utilisateurs uniques: {ratings['userId'].nunique():,}")    print(f"Période: {pd.to_datetime(ratings['timestamp'], unit='s').min()} à {pd.to_datetime(ratings['timestamp'], unit='s').max()}")    
    # Statistiques des ratings
    print("\n=== STATISTIQUES DES RATINGS ===")    print(ratings['rating'].describe())
    
    # Affichage des premières lignes
    print("\n=== APERÇU DES DONNÉES ===")display(ratings.head(10))

# Informations sur les types de données
print("\n=== INFORMATIONS SUR LES DONNÉES ===")print(ratings.info())

In [ ]:
# Distribution des ratings
if data and 'ratings' in data:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Distribution des ratings
    ratings['rating'].hist(bins=10, ax=axes[0,0], edgecolor='black')
    axes[0,0].set_title('Distribution des Ratings')
    axes[0,0].set_xlabel('Rating')
    axes[0,0].set_ylabel('Fréquence')
    
    # Nombre de ratings par utilisateur
    user_counts = ratings['userId'].value_counts()
    user_counts.hist(bins=50, ax=axes[0,1], edgecolor='black')
    axes[0,1].set_title('Distribution du Nombre de Ratings par Utilisateur')
    axes[0,1].set_xlabel('Nombre de Ratings')
    axes[0,1].set_ylabel('Nombre d\'Utilisateurs')
    axes[0,1].set_xscale('log')
    axes[0,1].set_yscale('log')
    
    # Nombre de ratings par film
    movie_counts = ratings['movieId'].value_counts()
    movie_counts.hist(bins=50, ax=axes[1,0], edgecolor='black')
    axes[1,0].set_title('Distribution du Nombre de Ratings par Film')
    axes[1,0].set_xlabel('Nombre de Ratings')
    axes[1,0].set_ylabel('Nombre de Films')
    axes[1,0].set_xscale('log')
    axes[1,0].set_yscale('log')
    
    # Évolution temporelle des ratings
    ratings['date'] = pd.to_datetime(ratings['timestamp'], unit='s')
    ratings['year'] = ratings['date'].dt.year
    yearly_counts = ratings['year'].value_counts().sort_index()
    yearly_counts.plot(ax=axes[1,1])
    axes[1,1].set_title('Évolution du Nombre de Ratings par Année')
    axes[1,1].set_xlabel('Année')
    axes[1,1].set_ylabel('Nombre de Ratings')
    
    plt.tight_layout()
    plt.show()

## 3. Exploration des Films

In [ ]:
if data and 'movies' in data:
    movies = data['movies']
    
    print("=== ANALYSE DES FILMS ===)
    print(f"Nombre total de films: {len(movies):,}")
    
    # Extraction de l'année depuis le titre
    movies['year'] = movies['title'].str.extract(r'\((\d{4})\)$')[0]
    movies['year'] = pd.to_numeric(movies['year'], errors='coerce')
    
    # Nettoyage du titre
    movies['clean_title'] = movies['title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)
    
    print(f"Période des films: {movies['year'].min():.0f} à {movies['year'].max():.0f}")
    
    # Affichage des premières lignes
    print("\n=== APERÇU DES FILMS ===)
    display(movies.head(10))
    
    # Analyse des genres
    print("\n=== ANALYSE DES GENRES ===)
    # Séparation des genres
    all_genres = []
    for genres in movies['genres'].dropna():
        all_genres.extend(genres.split('|'))
    
    genre_counts = pd.Series(all_genres).value_counts()
    print(f"Nombre de genres uniques: {len(genre_counts)}")
    print("\nTop 10 des genres:")
    print(genre_counts.head(10))

In [ ]:
# Visualisations des films
if data and 'movies' in data:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Distribution des années
    movies['year'].hist(bins=30, ax=axes[0,0], edgecolor='black')
    axes[0,0].set_title('Distribution des Films par Année')
    axes[0,0].set_xlabel('Année')
    axes[0,0].set_ylabel('Nombre de Films')
    
    # Top genres
    genre_counts.head(15).plot(kind='barh', ax=axes[0,1])
    axes[0,1].set_title('Top 15 des Genres')
    axes[0,1].set_xlabel('Nombre de Films')
    
    # Évolution des genres dans le temps
    top_genres = genre_counts.head(5).index
    for genre in top_genres:
        genre_by_year = []
        years = sorted(movies['year'].dropna().unique())
        for year in years:
            year_movies = movies[movies['year'] == year]
            genre_count = year_movies['genres'].str.contains(genre, na=False).sum()
            genre_by_year.append(genre_count)
        axes[1,0].plot(years, genre_by_year, label=genre, marker='o', markersize=2)
    
    axes[1,0].set_title('Évolution des Top 5 Genres dans le Temps')
    axes[1,0].set_xlabel('Année')
    axes[1,0].set_ylabel('Nombre de Films')
    axes[1,0].legend()
    
    # Nombre de genres par film
    movies['genre_count'] = movies['genres'].str.count('\|') + 1
    movies['genre_count'].hist(bins=10, ax=axes[1,1], edgecolor='black')
    axes[1,1].set_title('Distribution du Nombre de Genres par Film')
    axes[1,1].set_xlabel('Nombre de Genres')
    axes[1,1].set_ylabel('Nombre de Films')
    
    plt.tight_layout()
    plt.show()

## 4. Analyse Croisée Ratings-Films

In [ ]:
# Fusion des données
if data and 'ratings' in data and 'movies' in data:
    # Fusion ratings et movies
    ratings_movies = ratings.merge(movies, on='movieId', how='left')
    
    print("=== ANALYSE CROISÉE ===)
    print(f"Données fusionnées: {len(ratings_movies):,} ratings")
    
    # Films les plus populaires
    popular_movies = ratings_movies.groupby(['movieId', 'title']).agg({
        'rating': ['count', 'mean']
    }).round(2)
    
    popular_movies.columns = ['num_ratings', 'avg_rating']
    popular_movies = popular_movies.reset_index()
    
    # Filtrer les films avec au moins 100 ratings
    popular_movies_filtered = popular_movies[popular_movies['num_ratings'] >= 100]
    
    print("\n=== TOP 10 FILMS LES PLUS POPULAIRES (>100 ratings) ===)
    top_popular = popular_movies_filtered.nlargest(10, 'num_ratings')
    display(top_popular)
    
    print("\n=== TOP 10 FILMS LES MIEUX NOTÉS (>100 ratings) ===)
    top_rated = popular_movies_filtered.nlargest(10, 'avg_rating')
    display(top_rated)

In [ ]:
# Analyse par genre
if data and 'ratings' in data and 'movies' in data:
    # Analyse des ratings par genre
    genre_ratings = []
    
    for idx, row in movies.iterrows():
        if pd.notna(row['genres']):
            genres = row['genres'].split('|')
            movie_ratings = ratings[ratings['movieId'] == row['movieId']]['rating']
            
            for genre in genres:
                for rating in movie_ratings:
                    genre_ratings.append({'genre': genre, 'rating': rating})
    
    genre_ratings_df = pd.DataFrame(genre_ratings)
    
    # Statistiques par genre
    genre_stats = genre_ratings_df.groupby('genre')['rating'].agg(['count', 'mean', 'std']).round(2)
    genre_stats = genre_stats.sort_values('mean', ascending=False)
    
    print("=== STATISTIQUES PAR GENRE ===)
    display(genre_stats.head(15))

In [ ]:
# Visualisations avancées
if data and 'ratings' in data and 'movies' in data:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Relation entre popularité et note moyenne
    axes[0,0].scatter(popular_movies['num_ratings'], popular_movies['avg_rating'], alpha=0.6)
    axes[0,0].set_xlabel('Nombre de Ratings')
    axes[0,0].set_ylabel('Note Moyenne')
    axes[0,0].set_title('Relation Popularité vs Note Moyenne')
    axes[0,0].set_xscale('log')
    
    # Distribution des notes moyennes par genre (top 10)
    top_genres_stats = genre_stats.head(10)
    top_genres_stats['mean'].plot(kind='barh', ax=axes[0,1])
    axes[0,1].set_title('Note Moyenne par Genre (Top 10)')
    axes[0,1].set_xlabel('Note Moyenne')
    
    # Évolution des ratings dans le temps
    monthly_ratings = ratings_movies.groupby(ratings_movies['date'].dt.to_period('M'))['rating'].mean()
    monthly_ratings.plot(ax=axes[1,0])
    axes[1,0].set_title('Évolution de la Note Moyenne dans le Temps')
    axes[1,0].set_xlabel('Date')
    axes[1,0].set_ylabel('Note Moyenne')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Heatmap des ratings par heure et jour de la semaine
    ratings_movies['hour'] = ratings_movies['date'].dt.hour
    ratings_movies['day_of_week'] = ratings_movies['date'].dt.day_name()
    
    # Créer une heatmap simplifiée
    hour_day_counts = ratings_movies.groupby(['day_of_week', 'hour']).size().unstack(fill_value=0)
    
    # Réorganiser les jours de la semaine
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    hour_day_counts = hour_day_counts.reindex(day_order)
    
    im = axes[1,1].imshow(hour_day_counts.values, cmap='YlOrRd', aspect='auto')
    axes[1,1].set_xticks(range(0, 24, 4))
    axes[1,1].set_xticklabels(range(0, 24, 4))
    axes[1,1].set_yticks(range(len(day_order)))
    axes[1,1].set_yticklabels(day_order)
    axes[1,1].set_title('Activité des Ratings par Heure et Jour')
    axes[1,1].set_xlabel('Heure')
    axes[1,1].set_ylabel('Jour de la Semaine')
    
    plt.tight_layout()
    plt.show()

## 5. Analyse de la Sparsité

In [ ]:
# Analyse de la sparsité de la matrice utilisateur-film
if data and 'ratings' in data:
    n_users = ratings['userId'].nunique()
    n_movies = ratings['movieId'].nunique()
    n_ratings = len(ratings)
    
    # Calcul de la sparsité
    total_possible_ratings = n_users * n_movies
    sparsity = (1 - (n_ratings / total_possible_ratings)) * 100
    
    print("=== ANALYSE DE LA SPARSITÉ ===)
    print(f"Nombre d'utilisateurs: {n_users:,}")
    print(f"Nombre de films: {n_movies:,}")
    print(f"Nombre de ratings: {n_ratings:,}")
    print(f"Ratings possibles: {total_possible_ratings:,}")
    print(f"Sparsité: {sparsity:.2f}%")
    print(f"Densité: {100-sparsity:.2f}%")
    
    # Distribution des utilisateurs actifs
    user_activity = ratings['userId'].value_counts()
    
    print("\n=== ACTIVITÉ DES UTILISATEURS ==="),
    print(f"Utilisateur le plus actif: {user_activity.max()} ratings")
    print(f"Utilisateur le moins actif: {user_activity.min()} ratings")
    print(f"Moyenne de ratings par utilisateur: {user_activity.mean():.1f}")
    print(f"Médiane de ratings par utilisateur: {user_activity.median():.1f}")
    
    # Distribution des films populaires
    movie_popularity = ratings['movieId'].value_counts()
    
    print("\n=== POPULARITÉ DES FILMS ===)
    print(f"Film le plus populaire: {movie_popularity.max()} ratings")
    print(f"Film le moins populaire: {movie_popularity.min()} ratings")
    print(f"Moyenne de ratings par film: {movie_popularity.mean():.1f}")
    print(f"Médiane de ratings par film: {movie_popularity.median():.1f}")

## 6. Recommandations pour le Système

In [ ]:
# Recommandations basées sur l'analyse
print("=== RECOMMANDATIONS POUR LE SYSTÈME ===)
print()

print("1. 📊 DONNÉES:")
if data and 'ratings' in data:
    print(f"   • Dataset volumineux: {len(ratings):,} ratings")
    print(f"   • Sparsité élevée: {sparsity:.1f}% - nécessite des techniques robustes")
    print(f"   • Période: {ratings_movies['year'].min():.0f}-{ratings_movies['year'].max():.0f} - données historiques riches")

print("\n2. 🎯 APPROCHES RECOMMANDÉES:")
print("   • Filtrage collaboratif: Exploiter les similarités utilisateur-film")
print("   • Filtrage par contenu: Utiliser les genres et métadonnées")
print("   • Système hybride: Combiner les deux approches")
print("   • Factorisation matricielle: SVD, NMF pour gérer la sparsité")

print("\n3. ⚠️ DÉFIS IDENTIFIÉS:")
print("   • Cold start: Nouveaux utilisateurs/films")
print("   • Sparsité: Peu d'interactions par utilisateur")
print("   • Scalabilité: Volume important de données")
print("   • Biais temporel: Évolution des préférences")

print("\n4. 🛠️ TECHNIQUES À IMPLÉMENTER:")
print("   • SVD (Singular Value Decomposition)")
print("   • NMF (Non-negative Matrix Factorization)")
print("   • KNN (K-Nearest Neighbors)")
print("   • TF-IDF pour le contenu textuel")
print("   • Similarité cosinus")

print("\n5. 📈 MÉTRIQUES D'ÉVALUATION:")
print("   • RMSE, MAE pour la précision")
print("   • Precision@K, Recall@K pour le ranking")
print("   • NDCG pour la qualité du classement")
print("   • Coverage, Diversity pour la variété")

print("\n6. 🔧 OPTIMISATIONS:")
print("   • Filtrage des utilisateurs/films peu actifs")
print("   • Normalisation des ratings")
print("   • Validation temporelle")
print("   • Cache pour les recommandations fréquentes")

## 7. Sauvegarde des Insights

In [ ]:
# Sauvegarde des statistiques importantes
insights = {
    'dataset_stats': {
        'total_ratings': len(ratings) if data and 'ratings' in data else 0,
        'unique_users': ratings['userId'].nunique() if data and 'ratings' in data else 0,
        'unique_movies': ratings['movieId'].nunique() if data and 'ratings' in data else 0,
        'sparsity_percent': sparsity if data and 'ratings' in data else 0,
        'rating_range': [ratings['rating'].min(), ratings['rating'].max()] if data and 'ratings' in data else [0, 0],
        'time_range': [str(ratings['date'].min()), str(ratings['date'].max())] if data and 'ratings' in data else ['', '']
    },
    'recommendations': {
        'filtering_thresholds': {
            'min_user_ratings': 20,
            'min_movie_ratings': 10
        },
        'models_to_implement': ['SVD', 'NMF', 'KNN', 'Content-based', 'Hybrid'],
        'evaluation_metrics': ['RMSE', 'MAE', 'Precision@K', 'Recall@K', 'NDCG@K']
    },
    'analysis_date': str(datetime.now())
}

# Sauvegarde
import json

with open('../data/processed/data_insights.json', 'w', encoding='utf-8') as f:
    json.dump(insights, f, indent=2, ensure_ascii=False)

print("✅ Insights sauvegardés dans data/processed/data_insights.json")
print("\n📋 RÉSUMÉ DE L'EXPLORATION:")
print(f"   • {insights['dataset_stats']['total_ratings']:,} ratings analysés")
print(f"   • {insights['dataset_stats']['unique_users']:,} utilisateurs uniques")
print(f"   • {insights['dataset_stats']['unique_movies']:,} films uniques")
print(f"   • Sparsité: {insights['dataset_stats']['sparsity_percent']:.1f}%")
print("   • Prêt pour la phase de modélisation! 🚀")